In [3]:
pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 297 kB 3.7 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, random_split

2024-04-21 23:00:27.918259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 23:00:33.682894: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
!nvidia-smi


Sun Apr 21 23:00:40 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 PCIe    On   | 00000000:B5:00.0 Off |                    0 |
| N/A   39C    P0    52W / 350W |      0MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import numpy as np
import gc

In [5]:
os.listdir('/projects/academic/courses/cse676s24/bhanucha')

['logs', 'logsh', 'results', 'resultsh', 'train_data_v2.npy']

In [6]:
train_data_path = '/projects/academic/courses/cse676s24/bhanucha/train_data_v2.npy'
train_data = np.load(train_data_path, mmap_mode='r')

In [7]:
print("Data type:", type(train_data))
print("Data shape:", train_data.shape)
if isinstance(train_data, np.ndarray) and train_data.dtype.names is not None:
    print("Data field names:", train_data.dtype.names)
else:
    print("Contents of the array:", train_data[:1])

Data type: <class 'numpy.memmap'>
Data shape: (1405634, 512)
Contents of the array: [[    0 48539    35    22   134   740     4 10523  6515  6219  4696  1297
     22   134    73   176   740     4 27805  1070  5803  1297    22   134
     73   176 26141     4 21857  1297    22   134    73   176   740     4
   3187 15092    36 26512  1253 45894    22   176   255 39596     4  9050
     50 31417 27323  1297    22   246   112    73   176   740     4 10970
   1836 30274  7666 31729   113 13497    35   440    12   387  5113 14208
  41200 38490    35    22  1121    10  2016   132    12 45252  8929 12560
      6  3344  6219  4696     6 15092     6 27805  1070  5803     8  9050
     50 31417 27323 45863    22  5320   853    81  4761  2859   454 12652
  26054    70    81   299 45863    22   387 14189     8 14351   195   728
     55     4  4624   160  2859 45863    22  5320   853    11 21857     8
  25629   131  3344   157 45863    22 36949   132 37031     6  1874     8
   3989    88   389 28255   

In [8]:
class TokenizedDataset(Dataset):
    def __init__(self, numpy_data):
        self.input_ids = torch.tensor(numpy_data, dtype=torch.long)
        self.attention_mask = (self.input_ids != 1).long()

        # Assuming labels are the same as input_ids for an autoencoding task
        # If different, you would need to adjust this
        self.labels = torch.tensor(numpy_data, dtype=torch.long)

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]  # This line is critical
        }

    
dataset = TokenizedDataset(train_data)

# Split the dataset into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [9]:
# train_data = None
dataset = None
gc.collect() 

11

In [10]:
# Check GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")

Using GPU: NVIDIA H100 PCIe


In [11]:
model_checkpoint = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)


In [12]:

training_args = TrainingArguments(
    output_dir='/projects/academic/courses/cse676s24/bhanucha/results',
    num_train_epochs=1,
#     per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/projects/academic/courses/cse676s24/bhanucha/logs',
    logging_steps=10000,
    evaluation_strategy="epoch",
    save_strategy="steps",  
    save_steps=100000,  
    save_total_limit=2,  
    per_device_train_batch_size=4,  
    gradient_accumulation_steps=2,  
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.000000,0.000001


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=158134, training_loss=0.005809515866546127, metrics={'train_runtime': 16834.5077, 'train_samples_per_second': 75.147, 'train_steps_per_second': 9.393, 'total_flos': 3.856796506128384e+17, 'train_loss': 0.005809515866546127, 'epoch': 1.0})

In [14]:
model.save_pretrained('/projects/academic/courses/cse676s24/bhanucha/saved_model')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [17]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.000000,0.000001


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=158134, training_loss=0.00010101553865172577, metrics={'train_runtime': 16730.7583, 'train_samples_per_second': 75.613, 'train_steps_per_second': 9.452, 'total_flos': 3.86710105227264e+17, 'train_loss': 0.00010101553865172577, 'epoch': 1.0})

In [16]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 1.690886506366951e-06}


In [18]:
model.save_pretrained('/projects/academic/courses/cse676s24/bhanucha/saved_model2')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
